In [ ]:
#pip install pandas openpyxl

In [ ]:
import pandas as pd
import numpy as np
from datetime import timedelta
from openpyxl import load_workbook

# Чтение данных из файла
file_path = 'energo_gene3.xlsx'           # Имя входного файла
sheet_name='Выработка электроэнергии_мес'                       # Имя листа в книге
sheet_name_day='energo_gener_day1'         # Имя создаваемого  листа в книге с данными с дневным шагом
NameData_row='Data'                       # Имя столбца с Датами шаг месяц  должно быть в первой строке таблицы
NameVal_month='Eletro_Energy_Generation'  # Имя столбца с данными  

df = pd.read_excel(file_path, sheet_name=sheet_name, parse_dates=[NameData_row], thousands=',')

# Линейная интерполяция данных
data_daily = []
for i in range(len(df) - 1):
    start_date = df[NameData_row][i]
    end_date = df[NameData_row][i + 1]
    start_value = df[NameVal_month][i]
    end_value = df[NameVal_month][i + 1]

    days_difference = (end_date - start_date).days
    for j in range(days_difference):
        current_date = start_date + timedelta(days=j)
        weight = j / days_difference
        current_value = start_value + weight * (end_value - start_value)
        data_daily.append([current_date, current_value])

# Добавление последней строки
data_daily.append([df[NameData_row].iloc[-1], df[NameVal_month].iloc[-1]])

# Создание нового DataFrame
df_daily = pd.DataFrame(data_daily, columns=[NameData_row, NameVal_month])

# Запись нового листа в файл
with pd.ExcelWriter(file_path, engine='openpyxl', mode='a') as writer:
    book = load_workbook(file_path)
    writer.book = book
    df_daily.to_excel(writer, sheet_name=sheet_name_day, index=False)

print("Суточные данные успешно сохранены в файле.")


Суточные данные успешно сохранены в файле.
